In [9]:
# Auto-run data processing steps for ADMS-Urban outputs which were generated under Pasquill-Gifford conditions, for all regions

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from PyBNG import PyBNG
import shapely
from os import makedirs, path, listdir, remove
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import requests
import zipfile as zpf
from tqdm import tqdm
from PIL import Image
from ADMS_functions import PointXYZ_to_latlon, plot_on_map, plot_in_grid_box, process_PG_dataset

PG_index = ["A", "B", "C", "D", "E", "F", "G"]

runs = ["001", "003", "004", "005", "006", "007", "008", "009", "010"]

NaN_pcent_threshold = 9

for run in runs:
    print(f"\n\nRUN {run}")
    folder = f"/home/users/mwlw3/ADMS-Urban/2018_P-G_classes/all_regions/{run}/"
    files = [path.join(folder, file) for file in listdir(folder) if path.splitext(file)[-1]==".nc"]
    processed_coordinates_filepath = path.join(folder, "raw_processed_coordinates.nc")

    # Processing from raw ADMS-Urban outputs to a netCDF file with useful attributes and latitude/longitude coordinates
    if not path.exists(processed_coordinates_filepath):
        new_ds = xr.concat([process_PG_dataset(xr.open_dataset(file)) for file in files], "space")
        new_ds.to_netcdf(processed_coordinates_filepath)
    elif path.exists(processed_coordinates_filepath):
        new_ds = xr.open_dataset(processed_coordinates_filepath)
        print(f"Loaded the processed coordinate data for run {run}.")

    # Re-gridding the data to a latitude/longitude grid of highest relevant resolution
    print(f"Re-gridding run {run}...")
    xmin, ymin, xmax, ymax = gpd.points_from_xy(new_ds.longitude.values, 
                                                new_ds.latitude.values).total_bounds
    found_one = False
    n_cells = None
    ref_cell = None
    x_coords = None
    y_coords = None
    NaN_pcent_min = 100
    print("Searching for optimal re-gridding parameters...")
    for test_n_cells in tqdm(range(300, 1, -1)):
        cell_size = (xmax-xmin)/test_n_cells
        grid_cells = [shapely.geometry.box(x0, y0, x0 - cell_size, y0 + cell_size) 
                  for x0 in np.arange(xmin, xmax + cell_size, cell_size) 
                  for y0 in np.arange(ymin, ymax + cell_size, cell_size)]
        test_ref_cell = gpd.GeoDataFrame(grid_cells, columns=["geometry"])
        test_x_coords = test_ref_cell.centroid.x.round(12).drop_duplicates()
        test_y_coords = test_ref_cell.centroid.y.round(12).drop_duplicates()
        if len(test_ref_cell) == len(test_x_coords)*len(test_y_coords):
            variable = "NO2"
            i = 0
            # Grid the timeseries data
            cell_list = []
            cell = test_ref_cell.copy()
            class_gdf = gpd.GeoDataFrame(new_ds[variable][i, :].values, 
                             columns=[f"class_{variable}"], 
                             geometry=gpd.points_from_xy(new_ds.longitude.values, new_ds.latitude.values))
            merge = gpd.sjoin(class_gdf, test_ref_cell, how="left", predicate="within")
            dissolve = merge.dissolve(by="index_right", aggfunc="mean")
            cell.loc[dissolve.index, f"class_{variable}"] = dissolve[f"class_{variable}"].values
            cell_list.append(cell[f"class_{variable}"].values.reshape(len(test_x_coords),len(test_y_coords)))
            # Stack the grids into a numpy array
            classes_gridded = np.stack(cell_list, axis=-1)
            NaN_percentage = ((np.sum(np.isnan(classes_gridded)) / (classes_gridded.shape[0] * classes_gridded.shape[1] * classes_gridded.shape[2]))*100)
            if NaN_percentage <= NaN_pcent_threshold:
                found_one = True
                n_cells = test_n_cells
                ref_cell = test_ref_cell
                x_coords = test_x_coords
                y_coords = test_y_coords
                break
            elif NaN_percentage < NaN_pcent_min and not NaN_percentage <= NaN_pcent_threshold:
                NaN_pcent_min = NaN_percentage
                n_cells = test_n_cells
                ref_cell = test_ref_cell
                x_coords = test_x_coords
                y_coords = test_y_coords
    if not found_one:
        print(f"Couldn't get data gaps below {NaN_pcent_threshold}%. Minimum achieved was {NaN_pcent_min.round(1)}%.")
        NaN_percentage = NaN_pcent_min

    print(f"Selected to re-grid with {n_cells} cells in the x direction, resulting in {NaN_percentage.round(1)}% NaN gaps in the data.")
    grid_name = f"gridded_{n_cells}"
    variables = [var for var in list(new_ds.data_vars) if "wind" not in var]

    for variable in variables:
        filepath = path.join(folder, grid_name, f"{variable}_PG_classes_grid.nc")
        if path.exists(filepath):
            print(f"{grid_name}/{variable}_PG_classes_grid.nc already exists.")
            continue
        print(f"Re-gridding run {run}, pollutant {variable}...")

        # Grid the timeseries data
        cell_list = []
        for i in range(0, new_ds.PG_class.shape[0]):
            cell = ref_cell.copy()
            class_gdf = gpd.GeoDataFrame(new_ds[variable][i, :].values, 
                             columns=[f"class_{variable}"], 
                             geometry=gpd.points_from_xy(new_ds.longitude.values, new_ds.latitude.values))
            merge = gpd.sjoin(class_gdf, ref_cell, how="left", predicate="within")
            dissolve = merge.dissolve(by="index_right", aggfunc="mean")
            cell.loc[dissolve.index, f"class_{variable}"] = dissolve[f"class_{variable}"].values
            cell_list.append(cell[f"class_{variable}"].values.reshape(len(x_coords),len(y_coords)))

        # Stack the grids into a numpy array
        classes_gridded = np.stack(cell_list, axis=-1)

        # Create the xarray dataset
        data_variables = {f"{variable}": (["longitude", "latitude", "PG_class"], classes_gridded, new_ds[variable].attrs)
                            }

        coords = {"longitude": (["longitude"], x_coords),
                    "latitude": (["latitude"], y_coords),
                 "PG_class": (["PG_class"], new_ds.PG_class.data)}

        attrs = new_ds.attrs

        classes_ds = xr.Dataset(data_vars=data_variables, coords=coords, attrs=attrs)

        # Save to a netCDF file
        if not path.exists(path.join(folder, grid_name)):
            makedirs(path.join(folder, grid_name))
        classes_ds.to_netcdf(filepath)
        print(f"Saved to {filepath}.")
        
print(f"\nFinished processing runs.")



RUN 001
Loaded the processed coordinate data for run 001.
Re-gridding run 001...


  0%|          | 0/299 [00:00<?, ?it/s]

Searching for optimal re-gridding paramters...


  1%|          | 3/299 [02:10<3:34:43, 43.53s/it]


Selected to re-grid with 297 cells in the x direction, resulting in 8.2% NaN gaps in the data.
gridded_297/NOx_PG_classes_grid.nc already exists.
gridded_297/NO2_PG_classes_grid.nc already exists.
gridded_297/PM10_PG_classes_grid.nc already exists.
gridded_297/PM2.5_PG_classes_grid.nc already exists.
gridded_297/CO2_PG_classes_grid.nc already exists.


RUN 003
Loaded the processed coordinate data for run 003.
Re-gridding run 003...


...

Selected to re-grid with 294 cells in the x direction, resulting in 8.1% NaN gaps in the data.
gridded_294/NOx_PG_classes_grid.nc already exists.
gridded_294/NO2_PG_classes_grid.nc already exists.
gridded_294/PM10_PG_classes_grid.nc already exists.
gridded_294/PM2.5_PG_classes_grid.nc already exists.
gridded_294/CO2_PG_classes_grid.nc already exists.


RUN 010
Loaded the processed coordinate data for run 010.
Re-gridding run 010...


  0%|          | 0/299 [00:00<?, ?it/s]

Searching for optimal re-gridding paramters...


  2%|▏         | 6/299 [01:29<1:13:05, 14.97s/it]

Selected to re-grid with 294 cells in the x direction, resulting in 8.1% NaN gaps in the data.
gridded_294/NOx_PG_classes_grid.nc already exists.
gridded_294/NO2_PG_classes_grid.nc already exists.
gridded_294/PM10_PG_classes_grid.nc already exists.
gridded_294/PM2.5_PG_classes_grid.nc already exists.
gridded_294/CO2_PG_classes_grid.nc already exists.
Finished processing runs.
